```
CREATE TABLE Team_Player_Season (
    team_player_season_id SERIAL PRIMARY KEY,
    season_year INT NOT NULL,
    player_id INT REFERENCES Players(player_id),
    team_id INT REFERENCES Teams(team_id)
);
```
This table is only for knowing which players were in which teams in which season
1. season_year -> we have to extract it from the date of match.

wait -> I think we can do the things in better way. Look at another pending table -
```
CREATE TABLE Match_Players (
    match_player_id SERIAL PRIMARY KEY,
    match_id INT REFERENCES Matches(match_id),
    team_id INT REFERENCES Teams(team_id),
    player_id INT REFERENCES Players(player_id),
    is_playing BOOLEAN DEFAULT TRUE
);
```

In this table we are writing everything that team_Season_player is saving except the season_year, so how about we just add that one attribute in this table and delete that table altogether, it will make things much better.

So, We have followed my advice, now, next work -
1. season_year will get from match date and match match id also from match table
2. Now, the player id that was playing in that match also from match table and team id same from this table.
But I am not sure, how we will filter and map this whole data.

1. matches CSV includes: match_id, match_date, team1_players, team2_players, team1_id, team2_id
2. players table includes: player_id, display_name

In [49]:
import pandas as pd
import psycopg2

In [50]:
conn = psycopg2.connect(
    dbname="ipl_dashboard",
    user="ipl_user",
    password="ipl_user_12246",  
    host="localhost",
    port="5432"
)

In [51]:
matches_df = pd.read_sql("SELECT match_id, date, team1_id, team2_id FROM matches", conn)
players_df = pd.read_sql("SELECT player_id, display_name, name FROM players", conn)

C:\Users\hp\AppData\Local\Temp\ipykernel_4380\1614711426.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  matches_df = pd.read_sql("SELECT match_id, date, team1_id, team2_id FROM matches", conn)
C:\Users\hp\AppData\Local\Temp\ipykernel_4380\1614711426.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  players_df = pd.read_sql("SELECT player_id, display_name, name FROM players", conn)


In [52]:
print(matches_df.head())

   match_id        date  team1_id  team2_id
0        16  2008-04-22      18.0      14.0
1        17  2010-03-18      17.0       6.0
2        18  2024-05-10      16.0       2.0
3        19  2008-04-18      17.0       NaN
4        20  2016-05-29      17.0       8.0


In [53]:
match_raw = pd.read_csv("../../data/Match_Info.csv")

In [54]:
match_raw['match_date'].dtype

dtype('O')

In [55]:
matches_df['date'].dtype

dtype('O')

In [56]:
matches_df['date'] = pd.to_datetime(matches_df['date']).dt.date
match_raw['match_date'] = pd.to_datetime(match_raw['match_date']).dt.date

In [57]:
print(matches_df['date'].head(3))
print(match_raw['match_date'].head(3))

0    2008-04-22
1    2010-03-18
2    2024-05-10
Name: date, dtype: object
0    2008-04-22
1    2010-03-18
2    2024-05-10
Name: match_date, dtype: object


In [58]:
match_data = matches_df.merge(
    match_raw[['match_date', 'team1_players', 'team2_players']],
    left_on='date',
    right_on='match_date',
    how='left'
)

In [59]:
match_data['joined'] = match_data['team1_players'].notnull()
print(match_data['joined'].value_counts())

joined
True    1737
Name: count, dtype: int64


In [60]:
print(match_data.head())

   match_id        date  team1_id  team2_id  match_date  \
0        16  2008-04-22      18.0      14.0  2008-04-22   
1        17  2010-03-18      17.0       6.0  2010-03-18   
2        18  2024-05-10      16.0       2.0  2024-05-10   
3        19  2008-04-18      17.0       NaN  2008-04-18   
4        20  2016-05-29      17.0       8.0  2016-05-29   

                                       team1_players  \
0  AC Gilchrist, Y Venugopal Rao, VVS Laxman, A S...   
1  MK Pandey, JH Kallis, RV Uthappa, V Kohli, R D...   
2  S Sandeep Warrier, B Sai Sudharsan, Shubman Gi...   
3  R Dravid, W Jaffer, V Kohli, JH Kallis, CL Whi...   
4  CH Gayle, V Kohli, AB de Villiers, KL Rahul, S...   

                                       team2_players  joined  
0  G Gambhir, V Sehwag, S Dhawan, Shoaib Malik, K...    True  
1  NV Ojha, MJ Lumb, DR Martyn, AA Jhunjhunwala, ...    True  
2  TU Deshpande, AM Rahane, R Ravindra, RD Gaikwa...    True  
3  SC Ganguly, BB McCullum, RT Ponting, DJ Hussey...    

In [61]:
def explode_players(df, team_col, team_id_col):
    exploded = df[[team_col, team_id_col, 'match_id', 'date']].copy()
    exploded['player_name'] = exploded[team_col].str.split(', ')
    exploded = exploded.explode('player_name')
    exploded = exploded.dropna(subset=['player_name'])
    exploded['season_year'] = pd.to_datetime(exploded['date']).dt.year
    exploded.rename(columns={team_id_col: 'team_id'}, inplace=True)
    return exploded[['match_id', 'team_id', 'player_name', 'season_year']]

In [62]:
team1_df = explode_players(match_data, 'team1_players', 'team1_id')
team2_df = explode_players(match_data, 'team2_players', 'team2_id')
all_players_df = pd.concat([team1_df, team2_df])

In [63]:
print(all_players_df.head())

   match_id  team_id      player_name  season_year
0        16     18.0     AC Gilchrist         2008
0        16     18.0  Y Venugopal Rao         2008
0        16     18.0       VVS Laxman         2008
0        16     18.0        A Symonds         2008
0        16     18.0        RG Sharma         2008


In [65]:
players_df['name_clean'] = players_df['name'].str.upper().str.strip()
players_df['display_name_clean'] = players_df['display_name'].str.upper().str.strip()

In [66]:
map_by_name = dict(zip(players_df['name_clean'], players_df['player_id']))
map_by_display = dict(zip(players_df['display_name_clean'], players_df['player_id']))

In [67]:
print("Before drop:", all_players_df.shape[0])

Before drop: 38787


In [68]:
# Clean player_name from all_players_df
all_players_df['player_name_clean'] = all_players_df['player_name'].str.upper().str.strip()

# First try display_name match
all_players_df['player_id'] = all_players_df['player_name_clean'].map(map_by_display)

# For any still unmapped, try name match
missing_mask = all_players_df['player_id'].isnull()
all_players_df.loc[missing_mask, 'player_id'] = all_players_df.loc[missing_mask, 'player_name_clean'].map(map_by_name)

In [69]:
missing_count = all_players_df['player_id'].isnull().sum()
print(f"❌ Still unmapped after double-checking both display_name and name: {missing_count}")
print(all_players_df[all_players_df['player_id'].isnull()]['player_name'].value_counts().head(10))

❌ Still unmapped after double-checking both display_name and name: 19926
player_name
RV Uthappa         318
SK Raina           305
AT Rayudu          297
KA Pollard         281
AB de Villiers     279
YK Pathan          272
Harbhajan Singh    249
G Gambhir          241
DJ Bravo           228
SR Watson          216
Name: count, dtype: int64


In [71]:
print(all_players_df.head())

   match_id  team_id      player_name  season_year  player_id  \
0        16     18.0     AC Gilchrist         2008        NaN   
0        16     18.0  Y Venugopal Rao         2008        NaN   
0        16     18.0       VVS Laxman         2008        NaN   
0        16     18.0        A Symonds         2008        NaN   
0        16     18.0        RG Sharma         2008      151.0   

  player_name_clean  
0      AC GILCHRIST  
0   Y VENUGOPAL RAO  
0        VVS LAXMAN  
0         A SYMONDS  
0         RG SHARMA  


In [72]:
# Step 1: Drop unmatched players
final_match_players = all_players_df.dropna(subset=['player_id', 'match_id', 'team_id']).copy()

# Step 2: Convert to integer
final_match_players['player_id'] = final_match_players['player_id'].astype(int)
final_match_players['match_id'] = final_match_players['match_id'].astype(int)
final_match_players['team_id'] = final_match_players['team_id'].astype(int)

# Step 4: Add is_playing column
final_match_players['is_playing'] = True

# Step 5: Select only relevant columns
final_match_players = final_match_players[['match_id', 'team_id', 'player_id', 'season_year', 'is_playing']]


In [73]:
print(final_match_players.head())

   match_id  team_id  player_id  season_year  is_playing
0        16       18        151         2008        True
1        17       17         89         2010        True
1        17       17        223         2010        True
2        18       16         68         2024        True
2        18       16         67         2024        True


In [74]:
final_match_players.to_csv('cleaned_match_players.csv', index=False)